In [1]:
from nfl import NFL
nfl = NFL().load()

In [2]:
# look around
nfl('NFC')

NFC
            div overall                    division                     \
                    win loss tie       pct      win loss tie       pct   
team                                                                     
DAL    NFC-East      12    5   0  0.705882        5    1   0  0.833333   
PHI    NFC-East      11    6   0  0.647059        4    2   0  0.666667   
NYG    NFC-East       6   11   0  0.352941        3    3   0       0.5   
WAS    NFC-East       4   13   0  0.235294        0    6   0       0.0   
DET   NFC-North      12    5   0  0.705882        4    2   0  0.666667   
GB    NFC-North       9    8   0  0.529412        4    2   0  0.666667   
MIN   NFC-North       7   10   0  0.411765        2    4   0  0.333333   
CHI   NFC-North       7   10   0  0.411765        2    4   0  0.333333   
TB    NFC-South       9    8   0  0.529412        4    2   0  0.666667   
NO    NFC-South       9    8   0  0.529412        4    2   0  0.666667   
ATL   NFC-South       7   10   0  0.411765        3    3   0       0.5   
CAR   NFC-South       2   15   0  0.117647        1    5   0  0.166667   
SF     NFC-West      12    5   0  0.705882        5    1   0  0.833333   
STL    NFC-West      10    7   0  0.588235        5    1   0  0.833333   
SEA    NFC-West       9    8   0  0.529412        2    4   0  0.333333   
ARI    NFC-West       4   13   0  0.235294        0    6   0       0.0   

     conference                     
            win loss tie       pct  
team                                
DAL           9    3   0      0.75  
PHI           7    5   0  0.583333  
NYG           5    7   0  0.416667  
WAS           2   10   0  0.166667  
DET           8    4   0  0.666667  
GB            7    5   0  0.583333  
MIN           6    6   0       0.5  
CHI           6    6   0       0.5  
TB            7    5   0  0.583333  
NO            6    6   0       0.5  
ATL           4    8   0  0.333333  
CAR           1   11   0  0.083333  
SF           10    2   0  0.833333  
STL           8    4   0  0.666667  
SEA           7    5   0  0.583333  
ARI           3    9   0      0.25

In [3]:
nfl('MIN').schedule

,opp,at_home,score,opp_score,wlt
week,,,,,
1,TB,1,17,20,loss
2,PHI,0,28,34,loss
3,SD,1,24,28,loss
4,CAR,0,21,13,win
5,KC,1,20,27,loss
6,CHI,0,19,13,win
7,SF,1,22,17,win
8,GB,0,24,10,win
9,ATL,0,31,28,win


Much of the script is designed to help deconstruct the NFL's tiebreaking
procedures, but you can quickly get the playoff seeds as shown below.

In [4]:
# Report playoff seeds, in order
nfl('NFC').playoffs()

SF      NFC-West
DAL     NFC-East
DET    NFC-North
TB     NFC-South
PHI     Wildcard
STL     Wildcard
GB      Wildcard
Name: (div, ), dtype: object

The rest of this notebook gives examples of how to drill down into the tiebreaking procedures.

### Understanding tiebreakers ###

Since standings are sorted by division rank you can use pandas
to quickly filter the division champs
note the standings table is MultiIndexed which is why groupby takes a tuple

In [5]:
division_champs = nfl('NFC').standings.reset_index().groupby(('div','')).first()['team']
division_champs

(div, )
NFC-East     DAL
NFC-North    DET
NFC-South     TB
NFC-West      SF
Name: team, dtype: object

You can rank the division champs. The returned series provides both
the order and the tiebreaker basis if applicable. See help text
for tiebreaks

In [6]:
nfl.tiebreaks(list(division_champs))

SF           winner
DAL    head-to-head
DET      conference
TB          overall
Name: eliminated-by, dtype: object

In [7]:
# Find the wildcard seeds (top 3 teams afer removing the champs)
nfl.tiebreaks(nfl('NFC').teams - set(division_champs))

PHI                              winner
STL                             overall
GB                              overall
SEA                    victory-strength
NO                           conference
MIN                             overall
ATL                        head-to-head
CHI    division-tiebreaker:common-games
NYG                             overall
WAS                             overall
ARI                        head-to-head
CAR                             overall
Name: eliminated-by, dtype: object

You can see the calculations behind any tiebreaker by comparing
2 or more teams. In this case, GB, SEA and NO all had 9-8 records

In [8]:
nfl.tiebreakers(['GB', 'NO', 'SEA'])

team                     GB                            NO              \
outcome                 win   loss  tie        pct    win   loss  tie   
overall                 9.0    8.0  0.0   0.529412    9.0    8.0  0.0   
head-to-head            1.0    0.0  0.0        inf    0.0    1.0  0.0   
conference              7.0    5.0  0.0   0.583333    6.0    6.0  0.0   
common-games            3.0    2.0  0.0   0.600000    3.0    2.0  0.0   
victory-strength       70.0   83.0  0.0   0.457516   52.0  101.0  0.0   
schedule-strength     137.0  152.0  0.0   0.474048  125.0  164.0  0.0   
conference-rank         NaN    NaN  NaN  13.000000    NaN    NaN  NaN   
overall-rank            NaN    NaN  NaN  26.000000    NaN    NaN  NaN   
conference-netpoints    NaN    NaN  NaN  32.000000    NaN    NaN  NaN   
overall-netpoints       NaN    NaN  NaN  33.000000    NaN    NaN  NaN   

team                               SEA                         
outcome                     pct    win   loss  tie        pct  
overall                0.529412    9.0    8.0  0.0   0.529412  
head-to-head                inf    0.0    0.0  0.0        inf  
conference             0.500000    7.0    5.0  0.0   0.583333  
common-games           0.600000    3.0    2.0  0.0   0.600000  
victory-strength       0.339869   60.0   93.0  0.0   0.392157  
schedule-strength      0.432526  148.0  141.0  0.0   0.512111  
conference-rank       14.000000    NaN    NaN  NaN   6.000000  
overall-rank          27.500000    NaN    NaN  NaN  10.000000  
conference-netpoints  43.000000    NaN    NaN  NaN   0.000000  
overall-netpoints     75.000000    NaN    NaN  NaN -38.000000

In [9]:
# That's a lot of information; the 'pct' columns are the vital part
tb = nfl.tiebreakers(['GB', 'NO', 'SEA']).xs('pct', level=1, axis=1)

# Sort the columns by rule order putting the highest-ranked team on the left
tb.sort_values(list(tb.index), axis=1, ascending=False)

team,GB,SEA,NO
overall,0.529412,0.529412,0.529412
head-to-head,inf,inf,inf
conference,0.583333,0.583333,0.500000
common-games,0.600000,0.600000,0.600000
victory-strength,0.457516,0.392157,0.339869
schedule-strength,0.474048,0.512111,0.432526
conference-rank,13.000000,6.000000,14.000000
overall-rank,26.000000,10.000000,27.500000
conference-netpoints,32.000000,0.000000,43.000000
overall-netpoints,33.000000,-38.000000,75.000000


Here you can see that New Orleans is eliminated by virtue of its conference
record. According to the NFL's tiebreaker procedure they drop out
and the comparison starts over with the remaining 2 teams,
and GB wins based on strength of victory

In [10]:

tb = nfl.tiebreakers(['GB', 'SEA']).xs('pct', level=1, axis=1)
tb.sort_values(list(tb.index), axis=1, ascending=False)

team,GB,SEA
overall,0.529412,0.529412
head-to-head,NaN,NaN
conference,0.583333,0.583333
common-games,0.500000,0.500000
victory-strength,0.457516,0.392157
schedule-strength,0.474048,0.512111
conference-rank,13.000000,6.000000
overall-rank,26.000000,10.000000
conference-netpoints,32.000000,0.000000
overall-netpoints,33.000000,-38.000000


The inf and NaN values indicate that  the teams didn't play
each other equally. You can look at a detailed record among the
teams to learn why. The result shows that GB and NO played each other
(once) but neither team played SEA, hence the "head-to-head" rule doesn't
apply in this context.

In [11]:
nfl.matrix(['GB', 'SEA', 'NO'])

,GB,SEA,NO
GB,NaN,0.0,1.0
SEA,0.0,NaN,0.0
NO,1.0,0.0,NaN
